# 3_Create_Switch_Data.ipynb

Take in a full, merged history of investment fund data and create data thatindicates when fund switches hapen.


In [45]:
# importing the librarIES
import os                           # For file processing.
import csv
import pandas as pd            # For dataframes.
from pathlib import Path
import pyodbc                       # For connecting to SQL Server
import datetime
import numpy
import psutil 
import gc
import hashlib

# importing matplotlib module
import matplotlib.pyplot as plt
plt.style.use('default')
  
# %matplotlib inline: only draw static
# images in the notebook
%matplotlib inline

## Set all runtime variables.

In [51]:
e = datetime.datetime.now()
print ("Current date and time = %s" % e)
print ("Current memory available = ",psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

# File names
# Set values for the various paths.
merged_path = ".\data\merged"

# Set values for the various file extension types.
out_file_type = ".csv"

# Set values for the various file prefixes.
merged_file_prefix = "Policy_Fund_Agg_"

# Input file name
Full_History_Filename = r'C:/Users/m033\MSc\Big_Data_Project\data\output/Full_Fund_Investment_History.csv'

# Output file name
Switches_By_Month = r'C:/Users/m033\MSc\Big_Data_Project\data\azure/Switches_By_Month.csv'


Current date and time = 2023-04-22 13:39:06.785373
Current memory available =  34.79351279050326


# 1. Read back in the full history of switches.

In [52]:
# Read the data into a pandas dataframe
# File names
df_switches = pd.read_csv(Full_History_Filename, delimiter=',',low_memory=False, header=0)
df_switches

,POL_NUMB,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,API_TRANCHE_EXT,202303
0,001045160#,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,132935.50,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.80,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,7669.30,TSESS
3,001047062#,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,2556.40,TSDGB
4,001049026#,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,2556.40,TSESD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265863,8157682K,x,x,x,x,x,x,x,x,x,x,x,x,x,x,6768.00,XBEAAXBEAS
265864,8157702V,x,x,x,x,x,x,x,x,x,x,x,x,x,x,1200.00,5024A
265865,8157705J,x,x,x,x,x,x,x,x,x,x,x,x,x,x,2400.00,XBEAAXBEAS
265866,8157728W,x,x,x,x,x,x,x,x,x,x,x,x,x,x,2802.96,5024A


# 2. Compare the months one by one to identify where the summed funds change, indicating a switch.

In [53]:
df_switches['2022-02'] = df_switches['202201'].ne(df_switches['202202']) 
df_switches['2022-03'] = df_switches['202202'].ne(df_switches['202203']) 
df_switches['2022-04'] = df_switches['202203'].ne(df_switches['202204']) 
df_switches['2022-05'] = df_switches['202204'].ne(df_switches['202205']) 
df_switches['2022-06'] = df_switches['202205'].ne(df_switches['202206']) 
df_switches['2022-07'] = df_switches['202206'].ne(df_switches['202207']) 
df_switches['2022-08'] = df_switches['202207'].ne(df_switches['202208']) 
df_switches['2022-09'] = df_switches['202208'].ne(df_switches['202209']) 
df_switches['2022-10'] = df_switches['202209'].ne(df_switches['202210']) 
df_switches['2022-11'] = df_switches['202210'].ne(df_switches['202211']) 
df_switches['2022-12'] = df_switches['202211'].ne(df_switches['202212']) 
df_switches['2023-01'] = df_switches['202212'].ne(df_switches['202301']) 
df_switches['2023-02'] = df_switches['202301'].ne(df_switches['202302']) 
df_switches['2023-03'] = df_switches['202302'].ne(df_switches['202303']) 

# drop the index columns
df_switches.reset_index(drop=True, inplace=True)
df_switches.head(10)

,POL_NUMB,202201,202202,202203,202204,202205,202206,202207,202208,202209,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,001045160#,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,...,False,False,False,False,False,False,False,False,False,False
1,001045178#,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,...,False,False,False,False,False,False,False,False,False,False
2,001046154#,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,...,False,False,False,False,False,False,False,False,False,False
3,001047062#,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,...,False,False,False,False,False,False,False,False,False,False
4,001049026#,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,...,False,False,False,False,False,False,False,False,False,False
5,001050063#,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,...,False,False,False,False,False,False,False,False,False,False
6,001050203#,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,...,False,False,False,False,False,False,False,False,False,False
7,001052119#,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,...,False,False,False,False,False,False,False,False,False,False
8,001056186#,TUASC,TUASC,TUASC,TUASC,TUASC,TUASC,TUASC,x,x,...,False,False,True,False,False,False,False,False,False,False
9,001056204#,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,...,False,False,False,False,False,False,False,False,False,False


# 3. Remove all obsolete columns.
This step translates a list of funds per month into a timeline of fund switches.

In [54]:
# Remove obsolete columns
df_switches = df_switches.drop(columns=['202201', '202202','202203', '202204','202205', '202206','202207','202208',
                                        '202209', '202210', '202211', '202212', '202301', '202302', '202303'])
df_switches.head(10)

,POL_NUMB,API_TRANCHE_EXT,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,001045160#,132935.5,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,001045178#,17894.8,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,001046154#,7669.3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,001047062#,2556.4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,001049026#,2556.4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,001050063#,7669.2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,001050203#,20656.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,001052119#,7669.2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
8,001056186#,0.0,False,False,False,False,False,False,True,False,False,False,False,False,False,False
9,001056204#,15338.4,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# 4. Write the data out to file for future use.

In [55]:
# Write to a csv file and use index=false to drop teh index from the dataframe
df_switches.to_csv(Switches_By_Month, encoding='utf-8', index=False)